### Run VOLTTRON interface testing  


### Selecting a  Control and the Test Case 


In [18]:
import os
import sys
sys.path.append("..")

from Volttron_interface_os_c_1a_zone_temp_adj_inc_ratch import (
    VolttronControls,
    VolttronControlFunctions,
    VolttronInterface
)

# Select control and test case identifier 
control = 'os_c_1a_zone_temp_adj_inc_ratch' 

testcase = 'testcase_stanford' 
graph_path = os.path.join(os.getcwd(), f'{testcase}', f'{testcase}.ttl')

### Create Application 
Create application based on the selected control. 

In [19]:
from utility_functions.get_price_threshold import get_price_threshold

class TestCaseInterface(VolttronInterface):
    def __init__(self, controls, config_path):
        super().__init__(controls, config_path)
    
    def get_price_threshold(self, *args):
        return get_price_threshold(*args)
    
def create_application(config_path) -> VolttronInterface:
    '''takes a config file and returns an instantiated TestCaseInterface
    
    Parameters
    ----------
        config_path : str
            the path to a configuration file for instantiating a bestest air shed application 
    Returns
    -------
        application : TestCaseInterface
            returns an instantiated bestest air interface with shedding controls
'''
    
    functions = VolttronControlFunctions()
    controls = VolttronControls(functions)
    
    # Instantiate the TestCaseInterface with the controls, config_path, and hvac_signal_function
    application = TestCaseInterface(controls, config_path)
    return application


### Instantiate Application 
Instantiate the test case interface with the selected control.

In [20]:
import os
config_path = os.path.join(os.getcwd(), f'{testcase}', f'config_{control}.yml')

app = create_application(config_path)

# print of the control's SPARQL result

(rdflib.term.Literal('tes_berg'), rdflib.term.Literal('sensibo/tes_berg/ptac/temperature'), None, None, rdflib.term.Literal('occ_tes_berg'), None, None, None, None, None, None, None, rdflib.term.Literal('sensibo/tes_berg/ptac/targetTemperature'), None, None, None)
(rdflib.term.Literal('exp_berg'), rdflib.term.Literal('sensibo/exp_berg/ptac/temperature'), None, None, rdflib.term.Literal('occ_exp_berg'), None, None, None, None, None, None, None, rdflib.term.Literal('sensibo/exp_berg/ptac/targetTemperature'), None, None, None)


### Defining the Testing Parameters
Defining testing parameters for the first hour of the day

In [21]:
from datetime import datetime
now = datetime.now()

step = 1 # 1 for 1h; 0.25 for 15min and so on
current_time = now.hour + now.minute / 60  # Current hour + fraction of minutes
operation_mode = 'heat'

y = {
        'sensibo/tes_berg/ptac/temperature': 19,
        'sensibo/tes_berg/ptac/targetTemperature': 21,
        'sensibo/exp_berg/ptac/temperature': 19,
        'sensibo/exp_berg/ptac/targetTemperature': 21  
    }

output = app.control_agent(step, current_time, operation_mode, y)

print(f"Output at time {current_time}: {output}")

21.166666666666668
{'sensibo/tes_berg/ptac/temperature': 19, 'sensibo/tes_berg/ptac/targetTemperature': 21, 'sensibo/exp_berg/ptac/temperature': 19, 'sensibo/exp_berg/ptac/targetTemperature': 21}
{}
price_threshold_value 0.2666
range(0, 2) ['tes_berg', 'exp_berg'] ['sensibo/tes_berg/ptac/temperature', 'sensibo/exp_berg/ptac/temperature'] ['sensibo/tes_berg/ptac/targetTemperature', 'sensibo/exp_berg/ptac/targetTemperature'] [] [] [] [] [] [] [] [] ['occ_tes_berg', 'occ_exp_berg'] [] [] [] [] None None None
heat
tes_berg
occupancy_schedule [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
price_schedule [0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2666, 0.2666, 0.4, 0.6, 0.8, 0.9, 0.9666, 0.2666, 0.2666, 0.2383, 0.2383, 0.2383, 0.2383]
zone_set_temp_point []
occ_zone_set_temp_heat_point []
loop1
['sensibo/tes_berg/ptac/targetTemperature', 'sensibo/exp_berg/ptac/targetTempe

Run the control for the remaining hours of the day

In [22]:
total_hours = 24  # Run for 24 hours
total_steps = int(total_hours / step)  # 96 steps (24 * 4)

# Initialize previous values for both target temperatures
prev_target_temp_tes_berg = None
prev_target_temp_exp_berg = None

# Store outputs in a list
output_log = []

for i in range(total_steps): 
    current_time += step  # Increment by 15 minutes    

    if current_time >= 24:
        current_time -= 24
    
    # Get the target temperature values, default to the previous value if not available
    target_temp_tes_berg = output.get('sensibo/tes_berg/ptac/targetTemperature', prev_target_temp_tes_berg)
    target_temp_exp_berg = output.get('sensibo/exp_berg/ptac/targetTemperature', prev_target_temp_exp_berg)

    y = {
        'sensibo/tes_berg/ptac/temperature': 19,
        'sensibo/tes_berg/ptac/targetTemperature': target_temp_tes_berg,
        'sensibo/exp_berg/ptac/temperature': 19,
        'sensibo/exp_berg/ptac/targetTemperature': target_temp_exp_berg
    }

    output = app.control_agent(step, current_time, operation_mode, y)

    print(f"Output at time {current_time:.2f}: {output}")
    output_log.append(f"Output at time {current_time:.2f}: {output}")

    # Update previous values for the next iteration
    prev_target_temp_tes_berg = target_temp_tes_berg
    prev_target_temp_exp_berg = target_temp_exp_berg


22.166666666666668
{'sensibo/tes_berg/ptac/temperature': 19, 'sensibo/tes_berg/ptac/targetTemperature': 16.0, 'sensibo/exp_berg/ptac/temperature': 19, 'sensibo/exp_berg/ptac/targetTemperature': 16.0}
{0: 0, 1: 0}
price_threshold_value 0.2666
range(0, 2) ['tes_berg', 'exp_berg'] ['sensibo/tes_berg/ptac/temperature', 'sensibo/exp_berg/ptac/temperature'] ['sensibo/tes_berg/ptac/targetTemperature', 'sensibo/exp_berg/ptac/targetTemperature'] [] [] [] [] [] [] [] [] ['occ_tes_berg', 'occ_exp_berg'] [] [] [] [] None None None
heat
tes_berg
occupancy_schedule [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]
price_schedule [0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2666, 0.2666, 0.4, 0.6, 0.8, 0.9, 0.9666, 0.2666, 0.2666, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383]
zone_set_temp_point []
occ_zone_set_temp_heat_point []
loop1
['sensibo/tes_berg/ptac/targetTemperature', 'sensibo/exp_berg/pt

In [23]:
for log in output_log:
    print(log)

Output at time 22.17: {'sensibo/tes_berg/ptac/targetTemperature': 16.0, 'sensibo/exp_berg/ptac/targetTemperature': 16.0}
Output at time 23.17: {'sensibo/tes_berg/ptac/targetTemperature': 16.0, 'sensibo/exp_berg/ptac/targetTemperature': 16.0}
Output at time 0.17: {'sensibo/tes_berg/ptac/targetTemperature': 16.0, 'sensibo/exp_berg/ptac/targetTemperature': 16.0}
Output at time 1.17: {'sensibo/tes_berg/ptac/targetTemperature': 16.0, 'sensibo/exp_berg/ptac/targetTemperature': 16.0}
Output at time 2.17: {'sensibo/tes_berg/ptac/targetTemperature': 16.0, 'sensibo/exp_berg/ptac/targetTemperature': 16.0}
Output at time 3.17: {'sensibo/tes_berg/ptac/targetTemperature': 16.0, 'sensibo/exp_berg/ptac/targetTemperature': 16.0}
Output at time 4.17: {'sensibo/tes_berg/ptac/targetTemperature': 16.0, 'sensibo/exp_berg/ptac/targetTemperature': 16.0}
Output at time 5.17: {'sensibo/tes_berg/ptac/targetTemperature': 16.0, 'sensibo/exp_berg/ptac/targetTemperature': 16.0}
Output at time 6.17: {'sensibo/tes_ber